# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 10 - Mini-Projeto - Agrupando Clientes Por Consumo de Energia</font>

### Especificação

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pylab
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [38]:
#Maximo Colunas

pd.set_option('display.max_columns', None)

In [92]:
import pandas as pd
import sqlite3
import numpy as np


# Extraindo as tabelas do .sqlite
con = sqlite3.connect("Arquivo - Soccer/database.sqlite")
players = pd.read_sql_query("SELECT * FROM Player", con)
players_stats = pd.read_sql_query("SELECT * FROM Player_Attributes", con)
teams = pd.read_sql_query("SELECT * FROM Team", con)
league = pd.read_sql_query("SELECT * FROM League", con)
Matches = pd.read_sql_query("SELECT * FROM Match", con)

con.close()


In [93]:
#substituir caracteres e espaço por "_" no dataframe de Times

teams.team_long_name.replace(" ","_", regex=True, inplace=True)
teams.team_long_name.replace("-","_", regex=True, inplace=True)
teams.team_long_name.replace("\\.","_", regex=True, inplace=True)

In [94]:
#Merge de Matches, League e Teams

CountryClub = pd.merge_asof(Matches, league, on="country_id", by='country_id')
CountryClub = CountryClub.merge(teams, how='left', left_on="home_team_api_id", right_on="team_api_id")

In [95]:
#Merge entre Jogadores e Times

#Transformar as colunas de jogadores em uma coluna unica
tmp = Matches.melt(id_vars=['season','home_team_api_id'], 
               value_vars=['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5','home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11'],
                value_name='player_api_id')

#Excluir os valores nulos
tmp = tmp.dropna()

#Transformar os valores da planilha players em float
players = players.astype({"player_api_id": float,})

#Realizar o merge entre Matches e Players
left_sorted = tmp.sort_values(by="player_api_id")
right_sorted = players.sort_values(by="player_api_id")

tmp = pd.merge_asof(left_sorted, right_sorted, on="player_api_id")

#Realizar o merge entre Jogadores e Times
TeamsPlayer = tmp.merge(teams, how='left', left_on="home_team_api_id", right_on="team_api_id")

#Deixar em classificação alfabetica
TeamsPlayer = TeamsPlayer.sort_values('id_x')

#salvar em csv

TeamsPlayer.to_csv(r'C:\Users\Jefferson\OneDrive\Ciência de Dados\DSA\Big Data Analytics com R e Microsoft Azure Machine Learning\Manipulação de Arquivos TXT, CSV e Planilhas Excel\TeamsPlayer.txt', header=None, index=None, sep=' ', mode='a')



TypeError: Mismatch between array dtype ('object') and format specifier ('%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e,%.18e')

In [75]:
tmp.head()

,season,home_team_api_id,variable,player_api_id,id,player_name,player_fifa_api_id,birthday,height,weight
0,2011/2012,8021,home_player_6,2625.0,8372,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154
1,2009/2010,8569,home_player_8,2625.0,8372,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154
2,2012/2013,8569,home_player_7,2625.0,8372,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154
3,2011/2012,8021,home_player_7,2625.0,8372,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154
4,2009/2010,8569,home_player_8,2625.0,8372,"Patryk Rachwal,18",148544,1981-01-27 00:00:00,175.26,154


In [76]:
teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC_Genk,GEN
1,2,9993,675.0,Beerschot_AC,BAC
2,3,10000,15005.0,SV_Zulte_Waregem,ZUL
3,4,9994,2007.0,Sporting_Lokeren,LOK
4,5,9984,1750.0,KSV_Cercle_Brugge,CEB


In [88]:
TeamsPlayer.head()

,season,home_team_api_id,variable,player_api_id,id_x,player_name,player_fifa_api_id,birthday,height,weight,id_y,team_api_id,team_fifa_api_id,team_long_name,team_short_name
268236,2014/2015,8613,home_player_4,505942.0,1,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,38176,8613,1898.0,Boavista_FC,BOA
268235,2014/2015,8613,home_player_4,505942.0,1,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,38176,8613,1898.0,Boavista_FC,BOA
268234,2014/2015,8613,home_player_3,505942.0,1,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,38176,8613,1898.0,Boavista_FC,BOA
268233,2014/2015,8613,home_player_4,505942.0,1,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,38176,8613,1898.0,Boavista_FC,BOA
268232,2014/2015,8613,home_player_4,505942.0,1,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187,38176,8613,1898.0,Boavista_FC,BOA
